# Limpieza de los datos de la Base de datos `bookmaker.db`

Antes de empezar a desarrollar la limpieza de los datos, hemos seleccionado como Kernel el entorno que hemos creado previamente

In [2]:
%pip install numpy
%pip install matplotlib
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3

C:\Users\mglez\AppData\Local\Temp\ipykernel_8148\2253275120.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Ahora vamos a desglosar la base de datos en 6 tabblas, separadas unas de otras

In [ ]:
# Conectar a la base de datos SQLite
conn = sqlite3.connect("bookmaker.db")

# Listar las tablas en la base de datos
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(query, conn)

tables


,name
0,clientes
1,empresa
2,equipos
3,partidos
4,apuestas
5,cuotas


In [ ]:
# Cargar la tabla 'clientes' en un DataFrame
df_clientes = pd.read_sql("SELECT * FROM clientes", conn)
df_clientes.head()

,id,nombre,apellido,email,activado
0,1,Lope,Perales,pascualperera@vila.es,1
1,2,Isaura,Canet,gmerino@rocamora.es,1
2,3,Olga,Chaves,amayaprats@llado.com,1
3,4,Celestino,Hoyos,chedelgado@moraleda-pintor.com,1
4,5,Miguel,Milla,inigoprat@gmail.com,1


In [ ]:
# Cargar la tabla 'empresa' en un DataFrame
df_empresa = pd.read_sql("SELECT * FROM empresa", conn)
df_empresa.head()

,id,razon_social,email,activado


In [ ]:
# Cargar la tabla 'equipos' en un DataFrame
df_equipos = pd.read_sql("SELECT * FROM equipos", conn)
df_equipos.head()

,id,nombre,escudo,pais,puntaje,activado
0,1,Ajax,ajax.png,Paises Bajos,14.0,1
1,2,Atalanta,atalanta.png,Italia,6.0,1
2,3,Atlético,atletico.png,España,11.0,1
3,4,Barcelona,barcelona.png,España,17.0,1
4,5,Bayern,bayern.png,Alemania,16.0,1


In [ ]:
# Cargar la tabla 'partidos' en un DataFrame
df_partidos = pd.read_sql("SELECT * FROM partidos", conn)
df_partidos.head()

,id,fecha,equipo_local,equipo_visistante,finalizado,ganador
0,1,2021-12-15 00:51:23.181740,29,2,0,None
1,2,2021-12-15 00:51:23.181740,31,19,0,None
2,3,2021-12-15 00:51:23.181740,25,24,0,None
3,4,2021-12-15 00:51:23.181740,11,6,0,None
4,5,2021-12-15 00:51:23.181740,4,5,0,None


In [ ]:
# Cargar la tabla 'apuestas' en un DataFrame
df_apuestas = pd.read_sql("SELECT * FROM apuestas", conn)
df_apuestas.head()

,id,fecha,monto,equipo_ganador_id,partido,cliente,ganacia
0,1,2021-12-15 01:08:30.211674,300.0,2.0,1,88,891.0
1,2,2021-12-15 01:08:30.211674,300.0,29.0,1,88,891.0
2,3,2021-12-15 01:35:34.965516,300.0,2.0,1,88,891.0
3,4,2021-12-15 01:35:34.965516,300.0,29.0,1,88,891.0
4,5,2021-12-15 01:35:34.965516,300.0,2.0,1,88,891.0


In [ ]:
# Cargar la tabla 'cuotas' en un DataFrame
df_cuotas = pd.read_sql("SELECT * FROM cuotas", conn)
df_cuotas.head()

,id,local,empate,visitante,partido_id
0,1,1.40,2.97,5.80,1
1,2,5.73,5.08,3.86,2
2,3,5.45,5.99,5.34,3
3,4,1.95,5.08,4.78,4
4,5,4.44,5.99,4.61,5


Los valores de la columna `ganador`en la tabla `partidos` están vacios (llenos de None)

In [ ]:
df_partidos["ganador"].unique()

array([None], dtype=object)

En la tabla `apuestas` tambien hay valores nulos en la columna `equipo_ganador_id`

In [ ]:
df_apuestas["equipo_ganador_id"].unique()

array([ 2., 29., nan,  6.])

In [ ]:
print("None"), df_apuestas["equipo_ganador_id"].isnull().sum()

None


(None, 7)

## Explicación

1. En la tabla `apuestas`, los valores nulos en la columna `equipo_ganador_id` indican `apuestas` que se han realizado pero para las cuales el resultado del partido correspondiente aún no se ha determinado o no se ha registrado correctamente.

2. Los valores de la columna `ganador` en la tabla partidos están vacíos (llenos de None), lo cual sugiere que los resultados de algunos partidos no se han registrado.

## Soluciones propuestas:
1. __Para actualizar la columna equipo_ganador_id en apuestas:__ Necesitamos una manera de determinar el ganador del partido. Dado que la columna ganador en partidos está vacía, deberíamos buscar otra forma de deducir el ganador. Si no hay información directa sobre el resultado (por ejemplo, puntuaciones de los equipos en cada partido), esto podría complicarse.

2. __Para llenar la columna ganador en partidos:__ Si existe información indirecta (por ejemplo, en otra tabla o en un conjunto de reglas que podamos aplicar basándonos en los datos disponibles), podríamos utilizarla para actualizar esta columna.

## Insomnia Api

Por lo visto , Insomnia es una aplicación de escritorio gratuita que permite a los desarrolladores probar APIs. Facilita el envío de solicitudes HTTP y la organización de diferentes entornos para pruebas de APIs. Con Insomnia, puedes crear peticiones para probar métodos HTTP como GET, POST, PUT, DELETE, entre otros, así como configurar headers, añadir parámetros a las URLs, y enviar cuerpos de solicitud en varios formatos (como JSON o form-data). Esto lo hace especialmente útil para desarrollar y probar APIs REST y GraphQL.

Dentro de los datos proporcionados en el repositorio tenemos el archivo Insomnia_bookmaker, el cual vamos a utilizar para intentar solventar el problema con la falta de datos.

In [ ]:
# Importar la biblioteca necesaria para leer el archivo
import json

# Ruta al archivo subido
file_path = 'Insomnia_bookmaker'

# Leer y cargar el contenido del archivo JSON
with open(file_path, 'r') as file:
    data = json.load(file)

# Mostrar la estructura básica del archivo para entender su contenido
data.keys()


dict_keys(['_type', '__export_format', '__export_date', '__export_source', 'resources'])

El contenido del archivo indica que hay varias peticiones definidas, incluyendo información sobre URLs, métodos HTTP (como GET y POST), cuerpos de las peticiones, y headers. Por ejemplo, aquí hay algunos detalles extraídos:

### Ejemplo de petición

In [ ]:
import requests

# Realizar una petición GET
response = requests.get('http://127.0.0.1:8500/cuota/100')

# Verificar que la petición fue exitosa
if response.status_code == 200:
    print(response.json())  # Asumiendo que la respuesta es JSON
else:
    print(f'Error: {response.status_code}')


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8500): Max retries exceeded with url: /cuota/100 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002C09CCDBB10>: Failed to establish a new connection: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión'))